# Ingestion and Wrangling

In [1]:
#Importing Python packages
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

# INGESTION STEP

In [2]:
#Option 1: Read in the original csv files and save in memory

In [3]:
#Importing ACS csv data files 
fields = ["RT", "SERIALNO", "SPORDER","PWGTP","CIT","COW","DIVISION","PUMA","ST","REGION","AGEP","ADJINC","JWMNP","JWTR",
          "MAR","MARHT","MIG","MIL","RELP","SCHL","SEX","PAP","WAGP","WKHP","WRK","DIS","DRIVESP","ESR","FOD1P","HICOV","JWAP",
          "JWDP","MIGPUMA","MSP","NATIVITY","OC","OCCP","PAOC","PERNP","PINCP","RAC1P","SOCP","FHISP", "HISP", "NAICSP", "PWGTP"
         ]          

acs1 = pd.read_csv('data/psam_pusa.csv', sep=',', skipinitialspace=True, header=0, usecols=fields)
acs2 = pd.read_csv('data/psam_pusb.csv', sep=',', skipinitialspace=True, header=0, usecols=fields)

#print (acs1.head())
#print (acs2.head())

In [4]:
acs1.shape

(1638294, 45)

In [5]:
acs2.shape

(1551746, 45)

In [6]:
#Concatenate Data Frames to create ACSdata

ACSdata = pd.concat([acs1, acs2], axis=0, ignore_index=False)
print(ACSdata.head())
ACSdata.shape

  RT       SERIALNO  DIVISION  SPORDER  PUMA  REGION  ST   ADJINC  PWGTP  \
0  P  2017000000016         6        1  2500       3   1  1011189    206   
1  P  2017000000031         6        1  1800       3   1  1011189     45   
2  P  2017000000061         6        1  2400       3   1  1011189    136   
3  P  2017000000061         6        2  2400       3   1  1011189    121   
4  P  2017000000061         6        3  2400       3   1  1011189    111   

   AGEP  ...  NAICSP  NATIVITY   OC    OCCP  PAOC    PERNP    PINCP  RAC1P  \
0    73  ...     NaN         1  0.0     NaN   NaN      0.0  10000.0      2   
1    31  ...    6214         1  0.0   350.0   4.0  38500.0  38500.0      1   
2    41  ...      23         1  0.0  6260.0   NaN  72000.0  82000.0      1   
3    48  ...     NaN         1  0.0     NaN   2.0      0.0   8700.0      1   
4    16  ...     NaN         1  1.0     NaN   NaN      0.0      0.0      1   

     SOCP  FHISP  
0     NaN      0  
1  119111      0  
2  472061      0 

(3190040, 45)

# Merging State and PUMA codes to create unique location (GEOID)

In [7]:
#Function to normalize PUMA length (making sure that PUMA = len(5), adding leading zeroes where missing)
def normalize_PUMA(puma):
    try: 
        test = int(puma)
        puma_t = str(test)
    except:
        return prin('Error: not an integer')
    else:
        if len(puma_t) == 3:
            puma_t = '00' + str(puma_t)
        elif len(puma_t)== 4:
            puma_t = '0' + str(puma_t)
        elif len(puma_t)== 5:
            puma_t = puma_t
        else:
            puma_t = 'size error'
        return puma_t

In [8]:
#Create a new column to store the normalized Puma from say 100 to 00100
PUMA_T = []
for p in ACSdata['PUMA']:
    new_p = normalize_PUMA(p)
    PUMA_T.append(new_p)
ACSdata['PUMA_T'] = pd.DataFrame(PUMA_T)

In [9]:
#Print length of the transformed variable
print(len(PUMA_T))

3190040


In [10]:
#Print lenght of the new PUMA_T
print(len(ACSdata['PUMA_T']))

3190040


In [11]:
#Normalize the length ST due to possible issues with the leading zero
ST_T = []
ST_T =[ST if len(ACSdata['ST']) == 2 else '0'+str(ST) for ST in ACSdata['ST'] ]
ACSdata['ST_T'] = pd.DataFrame(ST_T)

In [12]:
#Create GEOID which is a combination of ST + PUMA
ACSdata['GEOID']= ACSdata['ST_T'].map(str) + ACSdata['PUMA_T'].map(str)
ACSdata['GEOID'].head(20)

0     0102500
1     0101800
2     0102400
3     0102400
4     0102400
5     0102500
6     0102500
7     0102500
8     0102500
9     0102500
10    0100500
11    0100500
12    0100500
13    0101900
14    0101900
15    0101900
16    0102701
17    0102703
18    0102703
19    0101100
Name: GEOID, dtype: object

# Creating a unique identifier for each row using SERIALNO and SPORDER (data_key_T)

In [13]:
#Checking for duplicates in the person records (grouped by household)

#How many unique households are in the dataset?
ACSdata['SERIALNO'].nunique()

1394399

In [14]:
#What is the min/max of people in any household?
ACSdata['SPORDER'].describe().astype('int64')

count    3190040
mean           2
std            1
min            1
25%            1
50%            2
75%            3
max           20
Name: SPORDER, dtype: int64

In [15]:
ACSdata['duplicate'] = ACSdata.duplicated(subset=['SPORDER', 'SERIALNO'], keep=False)
print('duplicate')

#Confirmed no duplicate SPORDERS in ACSdata

duplicate


In [16]:
#Create a unique identifier for each record
ACSdata['data_key_T'] = ACSdata['SERIALNO'].map(int).map(str) + ACSdata['SPORDER'].map(str)
ACSdata['data_key_T'].head()

0    20170000000161
1    20170000000311
2    20170000000611
3    20170000000612
4    20170000000613
Name: data_key_T, dtype: object

# Wrangling and Munging

In [17]:
#EXPLORING THE DATA

#Generating summary statistics - age
ACSdata['AGEP'].describe().astype('int64')

#Age range (0-96), data set needs to be filtered to include persons of marriageable age(> 18)

count    3190040
mean          41
std           23
min            0
25%           21
50%           42
75%           60
max           96
Name: AGEP, dtype: int64

In [18]:
#Rows in which value of AGEP column in  less than 18
is_lt18 = ACSdata.apply(lambda x: True if x['AGEP'] < 18 else False, axis=1)

#Count number of True in the series
numofRows = len(is_lt18[is_lt18 ==True].index)

print('Number of Rows in ACSdata in which Age < 18:', numofRows)

Number of Rows in ACSdata in which Age < 18: 659314


In [19]:
#Wrangling - Removing persons from the dataset who are < 18 years and creating a new df (ACSmarry)
ACSmarry = ACSdata[ACSdata.AGEP >= 18]
print(ACSmarry.shape)

(2530726, 50)


In [20]:
#EXPLORING THE DATAFRAME (ACSmarry)

In [21]:
#Exploring the marriage indicator
#Generating frequency table - marriage
ACSmarry['MAR'].value_counts(normalize=False)

1    1360300
5     671044
3     285664
2     169374
4      44344
Name: MAR, dtype: int64

In [22]:
#Exporting ACSmarry to csv
#ACSmarry.to_csv(r'data/check.csv')

In [23]:
# #Dropping rows where MAR not equal 1 or not equal 5 (cases where person is widowed, divorced or seperated)
# ACSmarry = ACSmarry[(ACSmarry.MAR ==1) | (ACSmarry.MAR ==5)]
ACSmarry.shape

#July 31, 2019 - After discussion with Maria and Mak, I reincorporated persons who are widowed, divorced or seperated

(2530726, 50)

In [24]:
# ACSmarry['MAR'].value_counts(sort=True) #Marital status

In [25]:
# #Transformaing--> Marrital Status (yes/no)
# ACSmarry.loc[ACSmarry.MAR == 5, 'MARRIED'] = 0
# ACSmarry.loc[ACSmarry.MAR == 1, 'MARRIED'] = 1

# ACSmarry['MARRIED'].value_counts(sort=True) 

In [26]:
#Maritual Status #2
ACSmarry.loc[(ACSmarry.MAR ==5) | (ACSmarry.MAR ==2) | (ACSmarry.MAR ==3), 'MARRIED'] = 0 #Not Married
ACSmarry.loc[(ACSmarry.MAR ==1) | (ACSmarry.MAR ==4), 'MARRIED'] = 1 #Married

ACSmarry['MARRIED'].value_counts(sort=True) 

C:\Users\steph\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\steph\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


1.0    1404644
0.0    1126082
Name: MARRIED, dtype: int64

In [27]:
#Using ADJINC to calculate wages/earning and income variables
#Note--> this code will need to be adjusted for the multi-year file

ACSmarry['WAGES'] = ACSmarry.WAGP * (ACSmarry.ADJINC / 1000000 )
ACSmarry['INCOME'] = ACSmarry.PINCP * (ACSmarry.ADJINC / 1000000 )
ACSmarry['EARN'] = ACSmarry.PERNP * (ACSmarry.ADJINC / 1000000 )

C:\Users\steph\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\steph\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\steph\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.

In [28]:
ACSmoney = ACSmarry[['WAGES', 'WAGP', 'INCOME', 'PINCP', 'EARN', 'PERNP']]

ACSmoney.describe().astype('int64')

,WAGES,WAGP,INCOME,PINCP,EARN,PERNP
count,2530726,2530726,2530726,2530726,2530726,2530726
mean,31072,30728,43202,42724,33157,32791
std,55479,54865,62819,62123,58403,57757
min,0,0,-9201,-9100,-9100,-9000
25%,0,0,9950,9840,0,0
50%,10213,10100,26290,26000,14156,14000
75%,43481,43000,53593,53000,45503,45000
max,744235,736000,1580488,1563000,1274098,1260000


In [29]:
#printing a few rows where income is < 0

filtered_data = ACSmarry[ACSmarry['INCOME'] < 0]
print(filtered_data[['INCOME', 'PINCP', 'EARN', 'PERNP']].head(100))

            INCOME   PINCP        EARN   PERNP
816     -364.02804  -360.0  -364.02804  -360.0
817     -364.02804  -360.0  -364.02804  -360.0
821    -1516.78350 -1500.0     0.00000     0.0
2484   -1516.78350 -1500.0     0.00000     0.0
4771   -1516.78350 -1500.0     0.00000     0.0
5218   -2831.32920 -2800.0 -2831.32920 -2800.0
8658   -5864.89620 -5800.0 -5864.89620 -5800.0
9275   -5864.89620 -5800.0 -5864.89620 -5800.0
9579   -5864.89620 -5800.0 -5864.89620 -5800.0
10334   -808.95120  -800.0  -808.95120  -800.0
13380  -1516.78350 -1500.0     0.00000     0.0
14314   -808.95120  -800.0  -808.95120  -800.0
15979  -5864.89620 -5800.0 -5864.89620 -5800.0
16373  -1314.54570 -1300.0     0.00000     0.0
17755  -1516.78350 -1500.0     0.00000     0.0
18600  -1516.78350 -1500.0     0.00000     0.0
22412  -5864.89620 -5800.0 -5864.89620 -5800.0
23407  -1516.78350 -1500.0     0.00000     0.0
23432  -5864.89620 -5800.0 -5864.89620 -5800.0
26746  -1516.78350 -1500.0     0.00000     0.0
27688  -1516.

In [30]:
#Persons with negative income or earning fall below the poverty line.  For ease of interpretation in the model, setting
#those values to 0(zero) in the dataframe using numpy

ACSmarry['INCOME'] = np.where(ACSmarry['INCOME'] < 0, 0, ACSmarry['INCOME'])
ACSmarry['EARN'] = np.where(ACSmarry['EARN'] < 0, 0, ACSmarry['EARN'])

C:\Users\steph\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\steph\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [31]:
ACSmarry.shape

(2530726, 54)

In [32]:
#Checking minimum values for INCOME and EARN
ACScheck = ACSmarry[['WAGES', 'WAGP', 'INCOME', 'PINCP', 'EARN', 'PERNP']]
ACScheck.describe().astype('int64')

,WAGES,WAGP,INCOME,PINCP,EARN,PERNP
count,2530726,2530726,2530726,2530726,2530726,2530726
mean,31072,30728,43205,42724,33161,32791
std,55479,54865,62816,62123,58400,57757
min,0,0,0,-9100,0,-9000
25%,0,0,9950,9840,0,0
50%,10213,10100,26290,26000,14156,14000
75%,43481,43000,53593,53000,45503,45000
max,744235,736000,1580488,1563000,1274098,1260000


In [33]:
#Frequency tables and possible transformation of categorical variables

#Citizenship status

ACSmarry['CIT'].value_counts(sort=True) 

1    2135403
4     197130
5     160061
3      23517
2      14615
Name: CIT, dtype: int64

In [34]:
#Transformaing--> Citizen (yes/no)
ACSmarry.loc[ACSmarry.CIT == 5, 'CITIZEN'] = 0
ACSmarry.loc[ACSmarry.CIT != 5, 'CITIZEN'] = 1

ACSmarry['CITIZEN'].value_counts(sort=True) 

1.0    2370665
0.0     160061
Name: CITIZEN, dtype: int64

In [35]:
#Transforming number of times married
ACSmarry['MARHT'].value_counts(sort=True) #Number of times married

1.0    1385866
2.0     366893
3.0     106923
Name: MARHT, dtype: int64

In [36]:
#Replacing blanks in MARHT with 0(zero)
ACSmarry['MARHT'].fillna(0, inplace = True)
ACSmarry['MARHT'].value_counts(sort=True)

C:\Users\steph\Anaconda3\lib\site-packages\pandas\core\generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


1.0    1385866
0.0     671044
2.0     366893
3.0     106923
Name: MARHT, dtype: int64

In [37]:
#Creating variable: Previously married (# of times)

ACSmarry.loc[ACSmarry.MARHT == 0, 'MAR_TIMES'] = 0 #Married zero times
ACSmarry.loc[(ACSmarry.MARHT == 1), 'MAR_TIMES'] = 1 #married 1 time
ACSmarry.loc[(ACSmarry.MARHT > 1), 'MAR_TIMES'] = 2 #married more than 1 times

ACSmarry['MAR_TIMES'].value_counts(sort=True) 

1.0    1385866
0.0     671044
2.0     473816
Name: MAR_TIMES, dtype: int64

In [38]:
#Mobility:  (lived here 1 year aga)
ACSmarry['MIG'].value_counts(sort=True)

1.0    2186818
3.0     329380
2.0      14528
Name: MIG, dtype: int64

In [39]:
#Transforming MIG to "Mover, or changed locations 1 year or less ago"
ACSmarry.loc[ACSmarry.MIG != 1, 'MOVER'] = 1 #Moved, changed location
ACSmarry.loc[ACSmarry.MIG == 1, 'MOVER'] = 0 #Did not move or change location

ACSmarry['MOVER'].value_counts(sort=True)

0.0    2186818
1.0     343908
Name: MOVER, dtype: int64

In [40]:
ACSmarry['MOVER'].value_counts(sort=True, normalize=True) * 100

0.0    86.410698
1.0    13.589302
Name: MOVER, dtype: float64

In [41]:
#Transforming educational attainment variable (SCHL)

ACSmarry['SCHL'].value_counts(sort=True)

16.0    593362
21.0    474195
19.0    385417
22.0    210462
20.0    208034
18.0    183853
17.0    100775
14.0     57847
23.0     51635
15.0     48886
13.0     41492
1.0      35862
24.0     34357
12.0     31612
11.0     26780
9.0      19603
10.0      8416
8.0       5445
6.0       4795
7.0       3696
5.0       2119
4.0        973
3.0        583
2.0        527
Name: SCHL, dtype: int64

In [42]:
ACSmarry.loc[ACSmarry.SCHL < 16, 'EDUCATION'] = 0 #No high school diploma or GED
ACSmarry.loc[(ACSmarry.SCHL == 16) | (ACSmarry.SCHL == 17), 'EDUCATION'] = 1 #HS diploma or GED
ACSmarry.loc[(ACSmarry.SCHL == 18) | (ACSmarry.SCHL == 19) | (ACSmarry.SCHL == 20), 'EDUCATION'] = 2 #less thhan a college degree or an associate's
ACSmarry.loc[(ACSmarry.SCHL == 21), 'EDUCATION'] = 3 #Bachelor's degree
ACSmarry.loc[(ACSmarry.SCHL == 22) | (ACSmarry.SCHL == 23) | (ACSmarry.SCHL == 24), 'EDUCATION'] = 4 #Graduate degree

ACSmarry['EDUCATION'].value_counts(sort=True)

2.0    777304
1.0    694137
3.0    474195
4.0    296454
0.0    288636
Name: EDUCATION, dtype: int64

In [43]:
#Marital status by educational attainment
pd.crosstab(ACSmarry.MARRIED, ACSmarry.EDUCATION, margins=True)

EDUCATION,0.0,1.0,2.0,3.0,4.0,All
MARRIED,,,,,,
0.0,152801,340995,373220,173312,85754,1126082
1.0,135835,353142,404084,300883,210700,1404644
All,288636,694137,777304,474195,296454,2530726


In [44]:
#JWTR: Means of Transportation to work
#Before Transformation
ACSmarry['JWTR'].value_counts(sort=True)

1.0     1231373
11.0      80443
10.0      40975
2.0       29522
4.0       25376
12.0      11953
5.0        8971
9.0        7865
8.0        2562
7.0        2372
3.0         772
6.0         641
Name: JWTR, dtype: int64

In [45]:
#Replacing blanks in JWTR with 0 (which indicates that person is not in the labor force)
ACSmarry['JWTR'].fillna(0, inplace = True)
ACSmarry['JWTR'].value_counts(sort=True)

1.0     1231373
0.0     1087901
11.0      80443
10.0      40975
2.0       29522
4.0       25376
12.0      11953
5.0        8971
9.0        7865
8.0        2562
7.0        2372
3.0         772
6.0         641
Name: JWTR, dtype: int64

In [46]:
# Transforming JWTR to WORK_SOC 
ACSmarry['WORK_SOC'] = ACSmarry['JWTR']
ACSmarry.loc[(ACSmarry.JWTR ==1)|(ACSmarry.JWTR ==8)|(ACSmarry.JWTR ==9)|(ACSmarry.JWTR ==11)|(ACSmarry.JWTR ==12),'WORK_SOC'] = 0
ACSmarry.loc[(ACSmarry.JWTR ==2)|(ACSmarry.JWTR ==3)|(ACSmarry.JWTR ==4)|(ACSmarry.JWTR ==5)|(ACSmarry.JWTR ==6)|(ACSmarry.JWTR ==7)|(ACSmarry.JWTR ==10),'WORK_SOC'] = 1
# 1 for JWTR: Bus, trolley car, Subway, Railroad, Ferryboat, Taxicab, Walk

C:\Users\steph\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [47]:
#After Transformation
ACSmarry['WORK_SOC'].value_counts(sort=True)

0.0    2422097
1.0     108629
Name: WORK_SOC, dtype: int64

In [48]:
#SEX: Transforming to 0/1
# Female = 0; Male =1
ACSmarry['SEX_T'] = ACSmarry['SEX']
ACSmarry.loc[(ACSmarry.SEX_T ==2),'SEX_T'] = 0

C:\Users\steph\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [49]:
#After Transformation
ACSmarry['SEX_T'].value_counts(sort=True)

0    1307012
1    1223714
Name: SEX_T, dtype: int64

In [50]:
#Before Transformation
ACSmarry['SEX'].value_counts(sort=True)

2    1307012
1    1223714
Name: SEX, dtype: int64

In [51]:
#Using ESR to created Employed/Unemployed
# Employed: 1,2,4,5 and unemployed 3,6
ACSmarry['ESR_T'] = ACSmarry['ESR']
ACSmarry.loc[(ACSmarry.ESR == 1)|(ACSmarry.ESR == 2)|(ACSmarry.ESR == 4)|(ACSmarry.ESR ==5),'ESR_T'] = 1
ACSmarry.loc[(ACSmarry.ESR == 3)|(ACSmarry.ESR ==6),'ESR_T'] = 0

C:\Users\steph\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [52]:
#After Transformation
ACSmarry['ESR_T'].value_counts(sort=True)

1.0    1472599
0.0    1058127
Name: ESR_T, dtype: int64

In [53]:
#Before Transformation
ACSmarry['ESR'].value_counts(sort=True)

1.0    1432375
6.0     985336
3.0      72791
2.0      29704
4.0      10450
5.0         70
Name: ESR, dtype: int64

In [54]:
#Nativity (Transformaing to 0/1)
# Foreign = 0; Native = 1
ACSmarry['NATIVITY_T'] = ACSmarry['NATIVITY']
ACSmarry.loc[(ACSmarry.NATIVITY == 2),'NATIVITY_T'] = 0

C:\Users\steph\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [55]:
#After Transformation
ACSmarry['NATIVITY_T'].value_counts(sort=True)

1    2173535
0     357191
Name: NATIVITY_T, dtype: int64

In [56]:
#Before Transformation
ACSmarry['NATIVITY'].value_counts(sort=True)

1    2173535
2     357191
Name: NATIVITY, dtype: int64

In [57]:
#Health insurance(Transformaing to 0/1)

ACSmarry['HEALTH'] = ACSmarry['HICOV'] 
ACSmarry.loc[(ACSmarry.HEALTH == 2),'HEALTH'] = 0 #No health insurance coverage
#1 = Health insurance coverage

ACSmarry['HEALTH'].value_counts(sort=True)

C:\Users\steph\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


1    2313875
0     216851
Name: HEALTH, dtype: int64

In [58]:
#Disability(Transformaing to 0/1)

ACSmarry['DIS_T'] = ACSmarry['DIS'] 
ACSmarry.loc[(ACSmarry.DIS_T == 2),'DIS_T'] = 0 #No disability
#1 = Disability

ACSmarry['DIS_T'].value_counts(sort=True)

C:\Users\steph\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


0    2090539
1     440187
Name: DIS_T, dtype: int64

# Recoding Race 

In [59]:
ACSmarry['RAC1P'].value_counts(sort=True, dropna = False)

1    1970142
2     249048
6     138466
8      87820
9      55103
3      21255
7       4030
5       3329
4       1533
Name: RAC1P, dtype: int64

In [60]:
ACSmarry['HISP'].value_counts(sort=True, dropna = False)

1     2205355
2      196338
3       31250
4       15518
24      14015
11      12055
5       10116
16       7898
7        7093
23       5736
8        4476
19       4249
17       4052
9        2824
21       2563
13       2008
10       1397
15       1114
6         987
14        736
20        439
12        187
18        185
22        135
Name: HISP, dtype: int64

In [61]:
pd.crosstab(ACSmarry.HISP, ACSmarry.RAC1P, margins=True)

RAC1P,1,2,3,4,5,6,7,8,9,All
HISP,,,,,,,,,,
1,1753887,242082,18633,1503,2020,137063,3649,4395,42123,2205355
2,132530,1284,1803,20,883,621,136,52917,6144,196338
3,20090,2112,159,0,103,143,92,6567,1984,31250
4,13727,550,13,1,14,50,16,759,388,15518
5,3885,1445,21,0,25,16,6,3976,742,10116
6,687,47,4,0,2,7,3,201,36,987
7,3681,63,130,0,23,16,23,2905,252,7093
8,2597,210,18,0,17,9,4,1465,156,4476
9,1887,49,18,0,6,28,1,712,123,2824


In [62]:
#Creating Race categories
ACSmarry['HISPANIC'] = 0
ACSmarry.loc[(ACSmarry.HISP > 1), 'HISPANIC'] = 1 #Hispanic = 1, else 0
             
ACSmarry['WHITE'] = 0
ACSmarry.loc[(ACSmarry.HISP ==1) & (ACSmarry.RAC1P == 1), 'WHITE'] = 1 #White= 1, else 0 
             
ACSmarry['BLACK'] = 0
ACSmarry.loc[(ACSmarry.HISP ==1) & (ACSmarry.RAC1P == 2), 'BLACK'] = 1 #Black= 1, else 0 

ACSmarry['INDIAN'] = 0
ACSmarry.loc[(ACSmarry.HISP ==1) & ((ACSmarry.RAC1P == 3) | (ACSmarry.RAC1P == 4) | (ACSmarry.RAC1P == 5)), 'INDIAN'] = 1 #NativeAmerican/Alaskan= 1, else 0 

ACSmarry['ASIAN'] = 0
ACSmarry.loc[(ACSmarry.HISP ==1) & ((ACSmarry.RAC1P == 6) | (ACSmarry.RAC1P == 7)), 'ASIAN'] = 1 #Asian/PacificIslander= 1, else 0 

ACSmarry['OTHER'] = 0
ACSmarry.loc[(ACSmarry.HISP ==1) & ((ACSmarry.RAC1P == 8) | (ACSmarry.RAC1P == 9)), 'OTHER'] = 1 #Other/TwoOrMore= 1, else 0 



C:\Users\steph\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\steph\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\steph\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#

In [63]:
ACSmarry['HISPANIC'].value_counts (dropna = False)

0    2205355
1     325371
Name: HISPANIC, dtype: int64

In [64]:
ACSmarry['WHITE'].value_counts (dropna = False)

1    1753887
0     776839
Name: WHITE, dtype: int64

In [65]:
ACSmarry['BLACK'].value_counts (dropna = False)

0    2288644
1     242082
Name: BLACK, dtype: int64

In [66]:
ACSmarry['INDIAN'].value_counts (dropna = False)

0    2508570
1      22156
Name: INDIAN, dtype: int64

In [67]:
ACSmarry['ASIAN'].value_counts (dropna = False)

0    2390014
1     140712
Name: ASIAN, dtype: int64

In [68]:
ACSmarry['OTHER'].value_counts (dropna = False)

0    2484208
1      46518
Name: OTHER, dtype: int64

# Mak will fix the OCCP code. Currently not working 7/30/2019

8/2/2019 -Having trouble running this code in this notebook.  Moved to a second notebook (MarriagePerson-Ingest_Wrangle_Pt2)

In [69]:
# Job Code Transformation function
#input must be interger data type 

# Job Code Transformation function
#input must be interger data type 

# def jobid_to_jobclass(occp):
#     try:
# #     try: 
        
# #         occp = int_math(occp)
# #         print(occp)
# # #         split_num = str(occp).split('.')
# # #         occp = int(split_num[0])
# #     except:
# #         return 'UNE'
# #     else:

#         if (occp >= 10 and occp <= 430):
#             jobclass = 'MGR'
#         elif occp <= 740:
#             jobclass = 'BUS'
#         elif occp <= 950:
#             jobclass = 'FIN'
#         elif occp <= 1240:
#             jobclass = 'CMM'
#         elif occp <= 1560:
#             jobclass = 'ENG'
#         elif occp <= 1965:
#             jobclass = 'SCI'
#         elif occp <= 2060:
#             jobclass = 'CMS'
#         elif occp <= 2160:
#             jobclass = 'LGL'
#         elif occp <= 2550:
#             jobclass = 'EDU'
#         elif occp <= 2920:
#             jobclass = 'ENT'
#         elif occp <= 1240:
#             jobclass = 'CMM'
#         elif occp <= 3540:
#             jobclass = 'MED'
#         elif occp <= 3655:
#             jobclass = 'HLS'
#         elif occp <= 3955:
#             jobclass = 'PRT'
#         elif occp <= 4150:
#             jobclass = 'EAT'
#         elif occp <= 4250:
#             jobclass = 'CMM'
#         elif occp <= 4650:
#             jobclass = 'PRS'
#         elif occp <= 4965:
#             jobclass = 'SAL'
#         elif occp <= 5940:
#             jobclass = 'OFF'
#         elif occp <= 6130:
#             jobclass = 'FFF'
#         elif occp <= 6765:
#             jobclass = 'CON'
#         elif occp <= 6940:
#             jobclass = 'EXT'
#         elif occp <= 7630:
#             jobclass = 'RPR'
#         elif occp <= 8965:
#             jobclass = 'PRD'
#         elif occp <= 9750:
#             jobclass = 'TRN'
#         elif occp <= 9830:
#             jobclass = 'MIL'
#         elif occp <= 9920:
#             jobclass = 'UNE' 
#         else:
#             jobclass = 'UNE'  
            
#         return jobclass
#     except:
#         return 'Error'

In [70]:
# def int_math(num):  
#     num = math.floor(num)
#     return num

In [71]:
# #Create List of integers to be converted in 3 Charatacter job code in the next cell
# OCCP_int = []
# #start = timeit.timeit()
# for i in ACSmarry['OCCP']:
#     try:
#         OCCP_int.append(int_math(i))
#     except: 
#          OCCP_int.append(9999)# if Cell is empty assume Unemployed


In [72]:
# #Create list of Job Codes of 3 Characters from the list of integers that represent job_id
# OCCP_list = []
# for i in OCCP_int:
#     try:
#         OCCP_list.append(jobid_to_jobclass(i))
#     except: 
#          OCCP_list.append(0)
# #print(OCCP_list)
# ACSmarry['OCCP_T'] = pd.DataFrame(OCCP_list)

In [73]:
# OCCP_T = []
# for i in ACSdata['OCCP']:
#     try:
#         OCCP_T.append(jobid_to_jobclass(i))
#     except: 
#          OCCP_T.append('UNE')
#     else:
#         ACSmarry['OCCP_T']= pd.DataFrame(OCCP_T)

In [74]:
# ACSmarry['OCCP_T'].value_counts(sort=True)

In [75]:
#Exporting ACSmarry to csv
#ACSmarry.to_csv(r'data/ACSmarry.csv')
ACSmarry.to_csv(r'data/ACSmarry.csv') #version 2

In [76]:
#End Of Code